# Example: WeightedSHAP on the fraud dataset

- In this notebook, we introduce weightedSHAP, which provides a generalized feature attribution method.

In [1]:
import sys, os
import numpy as np
import pickle 
np.random.seed(2022)
from analysis_utils import *

sys.path.append('../weightedSHAP')
import data, train, weightedSHAPEngine

## Load data
- We use the fraud dataset (https://www.openml.org/search?type=data&status=active&id=42397).
- A function `data.load_data` will load the `train`, `val`, `est`, and `test` datasets. 
 - `train`: to train a model to explain
 - `val`: to optimize hyperparameters
 - `est`: to estimate coalition functions
 - `test`: to evaluate the quality of feature attributions 

In [2]:
# Load dataset
problem='classification' 
dataset='fraud'
ML_model='boosting' 
(X_train, y_train), (X_val, y_val), (X_est, y_est), (X_test, y_test)=data.load_data(problem, dataset)    

------------------------------
Load a dataset
------------------------------
--------------------------------------------------
Fraud Detection
--------------------------------------------------
------------------------------
Before adding noise
Shape of X_train, X_val, X_est, X_test: (13310, 30), (1902, 30), (1902, 30), (1902, 30)
------------------------------
Rho: 0.0388
After adding noise
Shape of X_train, X_val, X_est, X_test: (13310, 90), (1902, 90), (1902, 90), (1902, 90)
------------------------------


## Train a model to explain
 - This step is a typical routine in machine learning. Given training and validation datasets, we train a model. Our goal is to interpret this model by looking a particular prediction (i.e., a local attribution problem).

In [3]:
model_to_explain=train.create_model_to_explain(X_train, y_train, X_val, y_val, problem, ML_model)

------------------------------
Train a model
Train a model to explain: Boosting
Training until validation scores don't improve for 25 rounds
Early stopping, best iteration is:
[12]	valid_0's binary_logloss: 0.646885	valid_0's binary_error: 0.0320715
Elapsed time for training a model to explain: 0.54 seconds
------------------------------


## Compute attributions and evaluate its performance
- `weightedSHAPEngine.run_attribution_core` computes conditional expectation values $\mathbb{E}[f(X) \mid X_S = x_S]$ based on various weighted versions $\phi_\mathbf{w} (x_i) := \sum_{j=1} ^{d} w_j \Delta_{j}(x_i)$ of SHAP (Equation (5) of the paper). We consider a set $\mathcal{W}$ which includes SHAP as well.
- We store them in `exp_dict` which includes
 - `cond_pred_keep_absolute` (add features with large absolute values first)
 - `cond_pred_remove_absolute` (remove features with large absolute values first)
 - `pred_masking` (add features with large absolute values first and other featuers are masked with zero).

In [ ]:
if not os.path.exists('fraud_example.pickle'):
    # Train a surrogate model and generate a coalition function
    # To efficiently estimate a conditional coalition function, we train a surrogate model. 
    conditional_extension=train.generate_coalition_function(model_to_explain, X_train, X_est, problem, ML_model)
    
    # With a surrogate model, we compute conditional expectations
    exp_dict=weightedSHAPEngine.run_attribution_core(problem, ML_model,
                                                     model_to_explain, conditional_extension,
                                                     X_train, y_train,
                                                     X_val, y_val, 
                                                     X_test, y_test)

    with open('fraud_example.pickle', 'wb') as handle:
        pickle.dump(exp_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open('fraud_example.pickle', 'rb') as handle:
        exp_dict = pickle.load(handle)


Elapsed time for training a surrogate model: 548.97 seconds


  0%|                                                                                                                | 0/100 [00:00<?, ?it/s]

Total number of random sets: 900, GR_stat: 1.007274776668469
Total number of random sets: 1000, GR_stat: 1.0067500997071723
Total number of random sets: 1100, GR_stat: 1.0066346271310442
Total number of random sets: 1200, GR_stat: 1.0051719850889846
Total number of random sets: 1300, GR_stat: 1.0059864735184305
Total number of random sets: 1400, GR_stat: 1.0055448719292726
Total number of random sets: 1500, GR_stat: 1.004794489707504
MCI score: 8994, Therehosld: 8991
We have seen 1600 random subsets for each feature.


  1%|█                                                                                                     | 1/100 [00:46<1:17:24, 46.92s/it]

Total number of random sets: 900, GR_stat: 1.007101462204402
Total number of random sets: 1100, GR_stat: 1.0064411589724716
Total number of random sets: 1300, GR_stat: 1.006964299222497
Total number of random sets: 1400, GR_stat: 1.005469781048714
Total number of random sets: 1500, GR_stat: 1.0050134224244465
Total number of random sets: 1600, GR_stat: 1.007086689745792
Total number of random sets: 1700, GR_stat: 1.0041482290171406
MCI score: 8996, Therehosld: 8991
We have seen 1800 random subsets for each feature.


...

 27%|███████████████████████████▊                                                                           | 27/100 [20:28<57:37, 47.36s/it]

Total number of random sets: 900, GR_stat: 1.010033082242018
Total number of random sets: 1100, GR_stat: 1.0070645976368044
Total number of random sets: 1500, GR_stat: 1.0061879406524161
Total number of random sets: 1600, GR_stat: 1.0041084704532455
MCI score: 8994, Therehosld: 8991
We have seen 1700 random subsets for each feature.


 28%|████████████████████████████▊                                                                          | 28/100 [21:18<57:40, 48.06s/it]

Total number of random sets: 1000, GR_stat: 1.0066044763084026
Total number of random sets: 1100, GR_stat: 1.00467821067648
MCI score: 8997, Therehosld: 8991
We have seen 1200 random subsets for each feature.


 29%|█████████████████████████████▊                                                                         | 29/100 [21:52<52:02, 43.98s/it]

## WeightedSHAP and Prediction recovery error curve 
 - We obtain weightedSHAP (Equation (6) of the paper)
 \begin{align*}
    \phi_{\mathrm{WeightedSHAP}}(\mathcal{T}, \mathcal{W}) := \phi_{\mathbf{w}^* (\mathcal{T}, \mathcal{W})},
 \end{align*} 
 where $\mathbf{w}^* (\mathcal{T}, \mathcal{W}) := \mathrm{argmax}_{\mathbf{w} \in \mathcal{W}} \mathcal{T} (\phi_{\mathbf{w}})$.
 - A default implementation for $\mathcal{T}$ includes the negative area under the prediction recovery error curve (AUP, Equation (4) of the paper) with `cond_pred_keep_absolute`. Note that `exp_dict` contains `remove_abolute` and `masking` as well.

In [ ]:
y_test=np.array(exp_dict['true_list'])
pred_list=np.array(exp_dict['pred_list'])
cond_pred_keep_absolute=np.array(exp_dict['cond_pred_keep_absolute'])
# Find a optimal weight and construct WeightedSHAP
optimal_ind_keep_absolute_list=find_optimal_list(cond_pred_keep_absolute, pred_list) 

# {13:MCI, 6:SHAP}
cond_pred_keep_absolute_short=np.array((cond_pred_keep_absolute[:,13,:],
                                    cond_pred_keep_absolute[:,6,:],
                                    cond_pred_keep_absolute[np.arange(len(optimal_ind_keep_absolute_list)),
                                                            optimal_ind_keep_absolute_list,:])).transpose((1,0,2))
recovery_curve_keep_absolute=np.mean(np.abs(cond_pred_keep_absolute_short - pred_list.reshape(-1,1,1)), axis=0)


 - The recovery error curves will become more smooth by increasing the number of test samples. It comes with more computational expenses. 

In [ ]:
n_features=len(recovery_curve_keep_absolute[0])
n_display_features=int(n_features*0.6)

plt.plot(recovery_curve_keep_absolute[0][max(1,int(n_features*0.075)):n_display_features],
         label='MCI', color='blue', linewidth=2, alpha=0.6)
plt.plot(recovery_curve_keep_absolute[1][max(1,int(n_features*0.075)):n_display_features],
         label='Shapley', color='green', linewidth=2, alpha=0.6)
plt.plot(recovery_curve_keep_absolute[2][max(1,int(n_features*0.075)):n_display_features],
         label='WeightedSHAP', color='red', linewidth=2, alpha=0.6)
plt.legend(fontsize=12)
xlabel_text='Number of features added' 
plt.title(f'Prediction recovery error curve, Dataset: fraud \n the lower, the better', fontsize=15)
plt.xticks(np.arange(n_features)[max(1,int(n_features*0.075)):n_display_features][::n_display_features//6],
               np.arange(n_features)[max(1,int(n_features*0.075)):n_display_features][::n_display_features//6])
plt.xlabel(xlabel_text, fontsize=15)
plt.ylabel(r'$|f(x)-\mathbb{E}[f(X) \mid X_S = x_S]|$', fontsize=15)
    